In [ ]:
!pip install geopandas
!pip install shapely

In [ ]:
# 새 셀에서 실행
try:
    import geopandas as gpd
    print("✅ GeoPandas 준비됨")
except ImportError:
    print("❌ GeoPandas 설치 필요")
    # conda install -c conda-forge geopandas

try:
    from shapely.geometry import Point
    print("✅ Shapely 준비됨")
except ImportError:
    print("❌ Shapely 설치 필요")

In [ ]:
# 새 셀에서 실행 - 각 데이터셋의 실제 컬럼명 확인
print("📋 각 데이터셋의 상권 관련 컬럼 확인:")
print("-" * 40)

datasets_check = {
    'boundary': boundary_df,
    'flow_latest': flow_latest, 
    'resident_latest': resident_latest,
    'sales_latest': sales_latest,
    'income_latest': income_latest,
    'facility_latest': facility_latest
}

for name, df in datasets_check.items():
    print(f"\n{name}:")
    # 상권 관련 컬럼 찾기
    district_cols = [col for col in df.columns if any(keyword in col 
                    for keyword in ['상권', 'TRDAR', '코드', 'CD', 'CODE'])]
    print(f"  상권 관련 컬럼: {district_cols}")
    
    # 첫 5개 컬럼만 표시
    print(f"  전체 컬럼 (앞 5개): {list(df.columns)[:5]}")

In [ ]:
# 1) 확인할 변수명 매핑 (키: 여러분이 부르고 싶은 이름, 값: 실제 변수명)
vars_to_check = {
    'boundary': 'boundary_df',
    'flow_latest': 'flow_latest',
    'resident_latest': 'resident_latest',
    'sales_latest': 'sales_latest',
    'income_latest': 'income_latest',
    'facility_latest': 'facility_latest'
}

# 2) 존재 여부, 타입, shape, columns 출력
for key, var_name in vars_to_check.items():
    if var_name in globals():
        df = globals()[var_name]
        try:
            rows, cols = df.shape
            print(f"✅ {key} ({var_name}): {type(df).__name__}, shape={rows}×{cols}")
            print("   columns:", df.columns.tolist())
        except Exception as e:
            print(f"⚠️ {key} ({var_name}): 변수는 존재하지만 .shape/.columns를 읽을 수 없습니다 ({e})")
    else:
        print(f"❌ {key}: 변수 `{var_name}` 가 정의되지 않았습니다.")


In [ ]:
# 새 셀에서 실행 - 개선된 상권코드 통일 함수
def standardize_district_code_fixed(df, dataset_name):
    """상권코드 컬럼을 TRDAR_CD로 통일 (실제 컬럼명 기반)"""
    
    # 데이터셋별 매핑 규칙
    column_mapping = {
        'boundary': '상권_코드',
        'flow': '상권_코드', 
        'resident': 'market_code',      # ← 이게 핵심!
        'sales': '상권_코드',
        'income': '상권_코드',
        'facility': '상권배후지_코드'    # ← 이것도!
    }
    
    target_col = column_mapping.get(dataset_name, '상권_코드')
    
    if target_col in df.columns:
        df['TRDAR_CD'] = df[target_col]
        print(f"✅ {dataset_name}: {target_col} → TRDAR_CD")
        return True
    else:
        # 백업 방법들
        possible_cols = ['TRDAR_CD', '상권_코드', 'market_code', '상권배후지_코드']
        for col in possible_cols:
            if col in df.columns:
                if col != 'TRDAR_CD':
                    df['TRDAR_CD'] = df[col]
                print(f"✅ {dataset_name}: {col} → TRDAR_CD (백업)")
                return True
    
    print(f"❌ {dataset_name}: 상권코드 컬럼을 찾을 수 없음")
    return False

# 테스트
print("🧪 상권코드 통일 테스트:")
standardize_district_code_fixed(resident_latest, 'resident')
standardize_district_code_fixed(facility_latest, 'facility')

In [ ]:
# 🏗️ 곱창집 입지 분석 마스터 데이터셋 구축
# 02_master_build.ipynb
# 
# 목적: 모든 데이터를 TRDAR_CD 기준으로 통합하여 
#       seoul_gopchang_master.gpkg 마스터 데이터셋 생성

# ================================================================================
# 📚 라이브러리 임포트
# ================================================================================

import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
from datetime import datetime

# 설정
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("🏗️ 곱창집 입지 분석 마스터 데이터셋 구축 시작!")
print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

# ================================================================================
# 📂 모든 데이터 로드
# ================================================================================

print("📂 모든 데이터 로드 중...")
print("-" * 30)

# 데이터 경로 설정
DATA_PATH = Path("../data")
RAW_PATH = DATA_PATH / "raw"

# 데이터 로드 상태 추적
data_status = {}

try:
    # 1. 상권 경계 데이터 (GIS 기준점)
    print("1️⃣ 상권 경계 데이터 로드...")
    boundary_df = pd.read_csv(DATA_PATH / "boundary/boundary_all.csv", encoding='utf-8')
    data_status['boundary'] = f"✅ 성공 ({boundary_df.shape})"
    print(f"   📍 상권 경계: {boundary_df.shape}")
    
    # 2. 유동인구 데이터 (야간 분석 완료)
    print("2️⃣ 유동인구 데이터 로드...")
    flow_df = pd.read_csv(RAW_PATH / "flow/flow_all.csv", encoding='utf-8')
    data_status['flow'] = f"✅ 성공 ({flow_df.shape})"
    print(f"   👥 유동인구: {flow_df.shape}")
    
    # 3. 상주인구 데이터
    print("3️⃣ 상주인구 데이터 로드...")
    resident_df = pd.read_csv(RAW_PATH / "resident/resident_all.csv", encoding='utf-8')
    data_status['resident'] = f"✅ 성공 ({resident_df.shape})"
    print(f"   🏠 상주인구: {resident_df.shape}")
    
    # 4. 추정매출 데이터
    print("4️⃣ 추정매출 데이터 로드...")
    sales_df = pd.read_csv(RAW_PATH / "sales/sales_all.csv", encoding='utf-8')
    data_status['sales'] = f"✅ 성공 ({sales_df.shape})"
    print(f"   💰 추정매출: {sales_df.shape}")
    
    # 5. 소득·소비 데이터
    print("5️⃣ 소득·소비 데이터 로드...")
    income_df = pd.read_csv(RAW_PATH / "income_exp/income_all.csv", encoding='utf-8')
    data_status['income'] = f"✅ 성공 ({income_df.shape})"
    print(f"   💳 소득·소비: {income_df.shape}")
    
    # 6. 집객시설 데이터
    print("6️⃣ 집객시설 데이터 로드...")
    facility_df = pd.read_csv(RAW_PATH / "facility/facility_all.csv", encoding='utf-8')
    data_status['facility'] = f"✅ 성공 ({facility_df.shape})"
    print(f"   🏢 집객시설: {facility_df.shape}")
    
except Exception as e:
    print(f"❌ 데이터 로드 오류: {e}")
    data_status['error'] = str(e)

print(f"\n📊 총 {len([k for k, v in data_status.items() if '✅' in v])}개 데이터셋 로드 완료")
print("\n" + "=" * 60)

# ================================================================================
# 🔍 데이터 구조 및 품질 분석
# ================================================================================

print("🔍 데이터 구조 및 품질 분석")
print("-" * 30)

# 각 데이터셋별 TRDAR_CD (상권코드) 분석
datasets = {
    'boundary': boundary_df,
    'flow': flow_df, 
    'resident': resident_df,
    'sales': sales_df,
    'income': income_df,
    'facility': facility_df
}

trdar_analysis = {}

for name, df in datasets.items():
    if 'TRDAR_CD' in df.columns or '상권_코드' in df.columns:
        # 상권코드 컬럼 찾기
        trdar_col = 'TRDAR_CD' if 'TRDAR_CD' in df.columns else '상권_코드'
        
        unique_codes = df[trdar_col].nunique()
        total_rows = len(df)
        latest_quarter = df['기준_년분기_코드'].max() if '기준_년분기_코드' in df.columns else 'N/A'
        
        trdar_analysis[name] = {
            'unique_districts': unique_codes,
            'total_rows': total_rows,
            'latest_quarter': latest_quarter,
            'trdar_column': trdar_col
        }
        
        print(f"📋 {name:10s}: {unique_codes:4d}개 상권, {total_rows:6d}행, 최신분기: {latest_quarter}")

print(f"\n🎯 기준 상권 수 (boundary): {trdar_analysis['boundary']['unique_districts']}개")

# 기준년분기 통일 분석
print(f"\n📅 기준년분기 분석:")
for name, df in datasets.items():
    if '기준_년분기_코드' in df.columns:
        quarters = sorted(df['기준_년분기_코드'].unique(), reverse=True)
        print(f"   {name:10s}: {quarters[:3]}... (총 {len(quarters)}개 분기)")

print("\n" + "=" * 60)

# ================================================================================
# ⚙️ 데이터 전처리 및 통합 준비
# ================================================================================

print("⚙️ 데이터 전처리 및 통합 준비")
print("-" * 30)

# 1. 최신 분기 데이터 추출 함수
def get_latest_quarter_data(df, name):
    """최신 분기 데이터만 추출"""
    if '기준_년분기_코드' not in df.columns:
        print(f"   {name}: 기준년분기 없음 (전체 데이터 사용)")
        return df
    
    latest_quarter = df['기준_년분기_코드'].max()
    latest_df = df[df['기준_년분기_코드'] == latest_quarter].copy()
    
    print(f"   {name}: {latest_quarter} → {len(latest_df):,}행")
    return latest_df

# 2. 최신 분기 데이터 추출
print("📅 최신 분기 데이터 추출:")

flow_latest = get_latest_quarter_data(flow_df, 'flow')
resident_latest = get_latest_quarter_data(resident_df, 'resident') 
sales_latest = get_latest_quarter_data(sales_df, 'sales')
income_latest = get_latest_quarter_data(income_df, 'income')
facility_latest = get_latest_quarter_data(facility_df, 'facility')

# 3. 야간 유동인구 계산 (이전 분석 결과 적용)
print(f"\n🌙 야간 유동인구 계산 (17-24시 기준):")

def calculate_night_flow(df):
    """17-24시 야간 유동인구 계산"""
    df = df.copy()
    
    # 17-24시 = 17-21시 + 21-24시
    df['야간_유동인구_17_24'] = (df['시간대_17_21_유동인구_수'] + 
                               df['시간대_21_24_유동인구_수'])
    
    # 추가 야간 지표들
    df['야간_비율'] = df['야간_유동인구_17_24'] / df['총_유동인구_수']
    df['주말_평균'] = (df['토요일_유동인구_수'] + df['일요일_유동인구_수']) / 2
    df['주중_평균'] = (df['월요일_유동인구_수'] + df['화요일_유동인구_수'] + 
                     df['수요일_유동인구_수'] + df['목요일_유동인구_수'] + 
                     df['금요일_유동인구_수']) / 5
    df['주말_프리미엄'] = df['주말_평균'] / df['주중_평균']
    
    return df

flow_latest = calculate_night_flow(flow_latest)
print(f"   ✅ 야간 유동인구 계산 완료")
print(f"   📊 평균 야간 유동인구: {flow_latest['야간_유동인구_17_24'].mean():,.0f}명")
print(f"   📊 평균 야간 비율: {flow_latest['야간_비율'].mean():.1%}")

print("\n" + "=" * 60)

# ================================================================================
# 🔗 마스터 데이터셋 구축 (단계별 조인)
# ================================================================================

print("🔗 마스터 데이터셋 구축 (단계별 조인)")
print("-" * 30)

# 1. 기준 데이터: 상권 경계 (boundary)
print("1️⃣ 기준 데이터 설정: 상권 경계")
master_df = boundary_df.copy()

# 상권코드 컬럼 통일
if '상권_코드' in master_df.columns:
    master_df['TRDAR_CD'] = master_df['상권_코드']

print(f"   📍 기준 상권 수: {len(master_df):,}개")

# 2. 유동인구 데이터 조인
print(f"\n2️⃣ 유동인구 데이터 조인:")
before_count = len(master_df)

# 상권코드 컬럼 통일 함수
def standardize_district_code(df, dataset_name):
    """상권코드 컬럼을 TRDAR_CD로 통일 (데이터셋별 매핑)"""
    
    # 데이터셋별 실제 컬럼명 매핑
    column_mapping = {
        'boundary': '상권_코드',
        'flow': '상권_코드', 
        'resident': 'market_code',      # resident는 market_code 사용
        'sales': '상권_코드',
        'income': '상권_코드',
        'facility': '상권배후지_코드'    # facility는 상권배후지_코드 사용
    }
    
    # 우선순위: 매핑된 컬럼 → 기본 후보들
    target_col = column_mapping.get(dataset_name, '상권_코드')
    fallback_cols = ['TRDAR_CD', '상권_코드', 'market_code', '상권배후지_코드', 'DISTRICT_CD']
    
    # 1차: 매핑된 컬럼 시도
    if target_col in df.columns:
        if target_col != 'TRDAR_CD':
            df['TRDAR_CD'] = df[target_col]
        print(f"   📋 {dataset_name} 상권코드: {target_col} → TRDAR_CD")
        return True
    
    # 2차: 백업 컬럼들 시도
    for col in fallback_cols:
        if col in df.columns:
            if col != 'TRDAR_CD':
                df['TRDAR_CD'] = df[col]
            print(f"   📋 {dataset_name} 상권코드: {col} → TRDAR_CD (백업)")
            return True
    
    print(f"   ❌ {dataset_name}: 상권코드 컬럼을 찾을 수 없음")
    print(f"   📋 사용 가능한 컬럼: {list(df.columns)[:10]}")
    return False

# 유동인구 데이터 상권코드 통일
if standardize_district_code(flow_latest, 'flow'):
    # 핵심 유동인구 컬럼만 선택
    flow_cols = ['TRDAR_CD', '총_유동인구_수', '야간_유동인구_17_24', '야간_비율',
                 '주말_평균', '주중_평균', '주말_프리미엄',
                 '남성_유동인구_수', '여성_유동인구_수',
                 '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수']
    
    # 실제 존재하는 컬럼만 선택
    available_flow_cols = [col for col in flow_cols if col in flow_latest.columns]
    flow_selected = flow_latest[available_flow_cols].copy()
    
    master_df = master_df.merge(flow_selected, on='TRDAR_CD', how='left')
    after_count = len(master_df)
    
    print(f"   📊 조인 전: {before_count:,}개 → 조인 후: {after_count:,}개")
    print(f"   📈 유동인구 매칭률: {(master_df['총_유동인구_수'].notna().sum() / len(master_df)):.1%}")
else:
    print(f"   ⚠️ 유동인구 데이터 조인 건너뜀")

# 3. 상주인구 데이터 조인
print(f"\n3️⃣ 상주인구 데이터 조인:")

if standardize_district_code(resident_latest, 'resident'):
    # 상주인구 관련 컬럼 찾기
    resident_pop_cols = [col for col in resident_latest.columns if '인구' in col]
    print(f"   📋 상주인구 관련 컬럼: {resident_pop_cols[:5]}")
    
    if resident_pop_cols:
        # 첫 번째 인구 관련 컬럼을 상주인구로 사용
        main_resident_col = resident_pop_cols[0]
        resident_latest['상주인구_수'] = resident_latest[main_resident_col]
        print(f"   📋 사용할 상주인구 컬럼: {main_resident_col}")
        
        resident_selected = resident_latest[['TRDAR_CD', '상주인구_수']].copy()
        master_df = master_df.merge(resident_selected, on='TRDAR_CD', how='left')
        
        print(f"   📈 상주인구 매칭률: {(master_df['상주인구_수'].notna().sum() / len(master_df)):.1%}")
    else:
        print(f"   ⚠️ 상주인구 컬럼을 찾을 수 없음")
else:
    print(f"   ⚠️ 상주인구 데이터 조인 건너뜀")

# 4. 추정매출 데이터 조인
print(f"\n4️⃣ 추정매출 데이터 조인:")

if standardize_district_code(sales_latest, 'sales'):
    # 추정매출 관련 컬럼 찾기
    sales_cols = [col for col in sales_latest.columns if '매출' in col or 'SAL' in col or 'SALES' in col]
    print(f"   📋 매출 관련 컬럼: {sales_cols[:5]}")
    
    if sales_cols:
        main_sales_col = sales_cols[0]
        sales_latest['추정매출'] = sales_latest[main_sales_col]
        print(f"   📋 사용할 매출 컬럼: {main_sales_col}")
        
        sales_selected = sales_latest[['TRDAR_CD', '추정매출']].copy()
        master_df = master_df.merge(sales_selected, on='TRDAR_CD', how='left')
        
        print(f"   📈 추정매출 매칭률: {(master_df['추정매출'].notna().sum() / len(master_df)):.1%}")
    else:
        print(f"   ⚠️ 매출 컬럼을 찾을 수 없음")
else:
    print(f"   ⚠️ 추정매출 데이터 조인 건너뜀")

# 5. 소득·소비 데이터 조인
print(f"\n5️⃣ 소득·소비 데이터 조인:")

if standardize_district_code(income_latest, 'income'):
    # 소득·소비 관련 컬럼 찾기
    income_cols = [col for col in income_latest.columns if '소득' in col or 'INCOME' in col]
    expense_cols = [col for col in income_latest.columns if '소비' in col or 'EXPEND' in col]
    
    print(f"   📋 소득 관련 컬럼: {income_cols[:3]}")
    print(f"   📋 소비 관련 컬럼: {expense_cols[:3]}")
    
    income_data = {}
    if income_cols:
        income_data['평균소득'] = income_latest[income_cols[0]]
    if expense_cols:
        income_data['평균소비'] = income_latest[expense_cols[0]]
    
    if income_data:
        income_selected = pd.DataFrame({'TRDAR_CD': income_latest['TRDAR_CD']})
        for key, series in income_data.items():
            income_selected[key] = series
        
        master_df = master_df.merge(income_selected, on='TRDAR_CD', how='left')
        print(f"   📈 소득·소비 매칭률: {(master_df[list(income_data.keys())[0]].notna().sum() / len(master_df)):.1%}")
    else:
        print(f"   ⚠️ 소득·소비 컬럼을 찾을 수 없음")
else:
    print(f"   ⚠️ 소득·소비 데이터 조인 건너뜀")

print(f"\n🎯 마스터 데이터셋 기본 구축 완료: {master_df.shape}")
print("\n" + "=" * 60)

# ================================================================================
# 📊 파생지표 생성
# ================================================================================

print("📊 곱창집 특화 파생지표 생성")
print("-" * 30)

# 1. 인구밀도 지표
print("1️⃣ 인구밀도 지표:")
master_df['유동밀도_점수'] = pd.qcut(master_df['총_유동인구_수'].fillna(0), 
                                  q=5, labels=[1,2,3,4,5]).astype(float)

master_df['야간밀도_점수'] = pd.qcut(master_df['야간_유동인구_17_24'].fillna(0), 
                                   q=5, labels=[1,2,3,4,5]).astype(float)

print(f"   ✅ 유동밀도 점수: 1(최저) ~ 5(최고)")
print(f"   ✅ 야간밀도 점수: 1(최저) ~ 5(최고)")

# 2. 매출 효율성 지표
print(f"\n2️⃣ 매출 효율성 지표:")
master_df['총인구'] = master_df['총_유동인구_수'].fillna(0) + master_df['상주인구_수'].fillna(0)

# 인구 1명당 추정매출
master_df['인당_매출효율'] = master_df['추정매출'] / master_df['총인구']
master_df['인당_매출효율'] = master_df['인당_매출효율'].replace([np.inf, -np.inf], np.nan)

# 야간 특화 효율성
master_df['야간_매출효율'] = master_df['추정매출'] / master_df['야간_유동인구_17_24']
master_df['야간_매출효율'] = master_df['야간_매출효율'].replace([np.inf, -np.inf], np.nan)

print(f"   ✅ 인당 매출효율: 평균 {master_df['인당_매출효율'].mean():.0f}원/명")
print(f"   ✅ 야간 매출효율: 평균 {master_df['야간_매출효율'].mean():.0f}원/명")

# 3. 곱창집 특화 지표
print(f"\n3️⃣ 곱창집 특화 지표:")

# 주류 친화 지표 (야간 + 주말 가중)
master_df['주류친화_지수'] = (master_df['야간_비율'].fillna(0) * 0.6 + 
                           master_df['주말_프리미엄'].fillna(1) * 0.4)

# 연령대별 타겟 고객 비율 (20-40대)
target_age_cols = ['연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수']
available_age_cols = [col for col in target_age_cols if col in master_df.columns]

if available_age_cols:
    master_df['타겟연령_비율'] = master_df[available_age_cols].sum(axis=1) / master_df['총_유동인구_수']
    master_df['타겟연령_비율'] = master_df['타겟연령_비율'].fillna(0)
    print(f"   ✅ 타겟연령(20-40대) 비율: 평균 {master_df['타겟연령_비율'].mean():.1%}")

# 남녀 비율 균형도 (곱창은 남녀 모두 선호)
if all(col in master_df.columns for col in ['남성_유동인구_수', '여성_유동인구_수']):
    master_df['성비_균형도'] = 1 - abs(master_df['남성_유동인구_수'] - master_df['여성_유동인구_수']) / master_df['총_유동인구_수']
    master_df['성비_균형도'] = master_df['성비_균형도'].fillna(0.5)
    print(f"   ✅ 성비 균형도: 평균 {master_df['성비_균형도'].mean():.1%}")

print(f"   ✅ 주류친화 지수: 평균 {master_df['주류친화_지수'].mean():.2f}")

print(f"\n📊 파생지표 생성 완료: {len([col for col in master_df.columns if '점수' in col or '지수' in col or '효율' in col])}개")
print("\n" + "=" * 60)

# ================================================================================
# 🗺️ 지리정보 처리 및 GeoPackage 생성
# ================================================================================

print("🗺️ 지리정보 처리 및 GeoPackage 생성")
print("-" * 30)

# 지리정보 컬럼 확인
geo_columns = [col for col in master_df.columns if any(keyword in col.lower() 
              for keyword in ['경도', 'lng', 'lon', '위도', 'lat', 'x좌표', 'y좌표'])]

print(f"📍 지리정보 컬럼: {geo_columns}")

if len(geo_columns) >= 2:
    # 경도/위도 컬럼 식별
    lng_col = next((col for col in geo_columns if '경도' in col or 'lng' in col.lower() or 'lon' in col.lower()), geo_columns[0])
    lat_col = next((col for col in geo_columns if '위도' in col or 'lat' in col.lower()), geo_columns[1])
    
    print(f"   📍 경도 컬럼: {lng_col}")
    print(f"   📍 위도 컬럼: {lat_col}")
    
    # 유효한 좌표 데이터 확인
    valid_coords = master_df[lng_col].notna() & master_df[lat_col].notna()
    valid_count = valid_coords.sum()
    
    print(f"   📊 유효 좌표: {valid_count:,}개 ({valid_count/len(master_df):.1%})")
    
    if valid_count > 0:
        # GeoDataFrame 생성
        geo_master = master_df[valid_coords].copy()
        
        try:
            from shapely.geometry import Point
            
            # Point 지오메트리 생성
            geometry = [Point(lng, lat) for lng, lat in zip(geo_master[lng_col], geo_master[lat_col])]
            geo_master_gdf = gpd.GeoDataFrame(geo_master, geometry=geometry, crs='EPSG:4326')
            
            print(f"   ✅ GeoDataFrame 생성 성공: {len(geo_master_gdf):,}개 상권")
            
            # GeoPackage 저장
            output_path = DATA_PATH / "processed"
            output_path.mkdir(exist_ok=True)
            
            gpkg_file = output_path / "seoul_gopchang_master.gpkg"
            geo_master_gdf.to_file(gpkg_file, driver='GPKG')
            
            print(f"   💾 GeoPackage 저장: {gpkg_file}")
            print(f"   📐 좌표계: EPSG:4326 (WGS84)")
            
        except Exception as e:
            print(f"   ❌ GeoDataFrame 생성 실패: {e}")
            geo_master_gdf = None
    else:
        print(f"   ⚠️ 유효한 좌표 데이터 부족")
        geo_master_gdf = None
else:
    print(f"   ⚠️ 지리정보 컬럼 부족 (CSV만 저장)")
    geo_master_gdf = None

# CSV 백업 저장
csv_file = DATA_PATH / "processed" / "seoul_gopchang_master.csv"
master_df.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"   💾 CSV 백업 저장: {csv_file}")

print("\n" + "=" * 60)

# ================================================================================
# 📊 마스터 데이터셋 품질 검증 및 요약
# ================================================================================

print("📊 마스터 데이터셋 품질 검증 및 요약")
print("-" * 30)

# 1. 기본 통계
print("1️⃣ 기본 통계:")
print(f"   📊 총 상권 수: {len(master_df):,}개")
print(f"   📋 총 컬럼 수: {len(master_df.columns)}개")

# 2. 핵심 지표별 결측률
print(f"\n2️⃣ 핵심 지표 결측률:")
key_columns = ['총_유동인구_수', '야간_유동인구_17_24', '상주인구_수', '추정매출']

for col in key_columns:
    if col in master_df.columns:
        missing_rate = master_df[col].isna().sum() / len(master_df)
        print(f"   {col:20s}: {missing_rate:6.1%} 결측")

# 3. 야간 유동인구 분포
print(f"\n3️⃣ 야간 유동인구 분포 (17-24시):")
if '야간_유동인구_17_24' in master_df.columns:
    night_stats = master_df['야간_유동인구_17_24'].describe()
    for stat, value in night_stats.items():
        if stat in ['mean', 'std', '50%', 'max']:
            print(f"   {stat:8s}: {value:10,.0f}명")

# 4. 상위 상권 미리보기
print(f"\n4️⃣ 야간 유동인구 TOP 10 상권:")
if '야간_유동인구_17_24' in master_df.columns and '상권_코드_명' in master_df.columns:
    top_districts = master_df.nlargest(10, '야간_유동인구_17_24')[['상권_코드_명', '야간_유동인구_17_24']]
    for i, (idx, row) in enumerate(top_districts.iterrows(), 1):
        print(f"   {i:2d}. {row['상권_코드_명']:20s}: {row['야간_유동인구_17_24']:7,.0f}명")

print("\n" + "=" * 60)

# ================================================================================
# 📈 데이터 품질 시각화
# ================================================================================

print("📈 마스터 데이터셋 품질 시각화")
print("-" * 30)

# 1. 결측률 히트맵
missing_rates = master_df.isnull().sum() / len(master_df)
important_cols = missing_rates[missing_rates > 0].head(15)

if len(important_cols) > 0:
    fig_missing = go.Figure(data=go.Bar(
        y=important_cols.index[::-1],  # 역순으로 표시
        x=important_cols.values[::-1] * 100,
        orientation='h',
        marker_color='lightcoral',
        text=[f'{val:.1f}%' for val in important_cols.values[::-1] * 100],
        textposition='auto'
    ))
    
    fig_missing.update_layout(
        title="📊 주요 컬럼별 결측률",
        xaxis_title="결측률 (%)",
        yaxis_title="컬럼명",
        height=500,
        showlegend=False
    )
    
    fig_missing.show()

# 2. 야간 유동인구 분포
if '야간_유동인구_17_24' in master_df.columns:
    night_flow_clean = master_df['야간_유동인구_17_24'].dropna()
    
    fig_dist = go.Figure()
    
    fig_dist.add_trace(go.Histogram(
        x=night_flow_clean,
        nbinsx=50,
        name='야간 유동인구',
        marker_color='lightblue',
        opacity=0.7
    ))
    
    fig_dist.update_layout(
        title="🌙 야간 유동인구 분포 (17-24시)",
        xaxis_title="야간 유동인구 (명)",
        yaxis_title="상권 수",
        height=400,
        showlegend=False
    )
    
    fig_dist.show()

# 3. 종합 대시보드
fig_summary = make_subplots(
    rows=2, cols=2,
    subplot_titles=('데이터 로드 현황', '핵심 지표 통계', '상위 상권 분포', '품질 점수'),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "table"}]]
)

# 3-1. 데이터 로드 성공률
success_data = [1 if '✅' in status else 0 for status in data_status.values() if '✅' in status or '❌' in status]
success_rate = sum(success_data) / len(success_data) if success_data else 0

fig_summary.add_trace(
    go.Pie(labels=['성공', '실패'], values=[success_rate, 1-success_rate],
           marker_colors=['lightgreen', 'lightcoral']),
    row=1, col=1
)

# 3-2. 핵심 지표 통계
key_stats = {}
for col in ['총_유동인구_수', '야간_유동인구_17_24', '상주인구_수']:
    if col in master_df.columns:
        key_stats[col.replace('_', ' ')] = master_df[col].mean()

if key_stats:
    fig_summary.add_trace(
        go.Bar(x=list(key_stats.keys()), y=list(key_stats.values()),
               marker_color='lightblue'),
        row=1, col=2
    )

# 3-3. 상권 분포 (지역별 - 상권구분코드 기준)
if '상권_구분_코드_명' in master_df.columns:
    district_counts = master_df['상권_구분_코드_명'].value_counts().head(10)
    
    fig_summary.add_trace(
        go.Scatter(x=list(range(len(district_counts))), y=district_counts.values,
                  mode='markers+lines', marker_size=10, marker_color='orange'),
        row=2, col=1
    )

# 3-4. 품질 요약 테이블
quality_summary = [
    ['총 상권 수', f'{len(master_df):,}개'],
    ['완전데이터 상권', f'{master_df.dropna().shape[0]:,}개'],
    ['야간데이터 보유', f'{master_df["야간_유동인구_17_24"].notna().sum():,}개'],
    ['평균 야간 유동인구', f'{master_df["야간_유동인구_17_24"].mean():,.0f}명'],
    ['지리정보 보유', f'{valid_count if "valid_count" in locals() else 0:,}개']
]

fig_summary.add_trace(
    go.Table(
        header=dict(values=['지표', '값'], fill_color='lightblue', align='center'),
        cells=dict(values=list(zip(*quality_summary)), fill_color='white', align='center')),
    row=2, col=2
)

fig_summary.update_layout(
    title="🎯 곱창집 마스터 데이터셋 종합 현황",
    height=800,
    showlegend=False
)

fig_summary.show()

print("\n" + "=" * 60)

# ================================================================================
# 🎯 완료 및 다음 단계 안내
# ================================================================================

print("🎯 마스터 데이터셋 구축 완료!")
print("-" * 30)

print(f"✅ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n📊 최종 결과:")
print(f"   🗃️  마스터 데이터셋: {master_df.shape}")
print(f"   🌙 야간 유동인구 (17-24시): {master_df['야간_유동인구_17_24'].notna().sum():,}개 상권")
print(f"   💾 저장 파일: seoul_gopchang_master.csv / .gpkg")
print(f"   📐 지리정보: {valid_count if 'valid_count' in locals() else 0:,}개 상권")

print(f"\n🎯 다음 단계 옵션:")
print(f"   A. KPI 스코어링 (경쟁밀도, 임대료, 통합점수)")
print(f"   B. 상권별 상세 분석 (TOP 30 추출)")
print(f"   C. 지도 시각화 (Folium 인터랙티브 맵)")
print(f"   D. 대시보드 구축 (Plotly Dash)")

print(f"\n💡 권장: A번 (KPI 스코어링)부터 진행")
print(f"   이유: 마스터 데이터가 완성되었으니 곱창집 특화 점수 계산 가능")

print("\n" + "=" * 60)
print("🏗️ 마스터 데이터셋 구축 완료!")
print("🚀 이제 본격적인 입지 분석을 시작할 준비가 되었습니다!")